# Preliminaries

**Load dependencies**

In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import pylab as py
import scipy

**Download data: 'black box' predictions by random forest**

In [ ]:
# URL of dataset - with predictions generated by linear regression
URL = "https://github.com/SHeidema/AMES-TRIDSA/blob/main/datasets/predictions_rf.csv?raw=true"

# Load dataset
df_full = (pd.read_csv(URL))

# Select first house, for which we will calculate a prediction interval
df_0 = df_full.iloc[0]

# Remove first house from full dataset
df = df_full.drop(index=0)

# Exploratory data analysis

Let $Y_i$ be the real sale price, and $\hat{Y_i}$ its predicted value. The residuals are defined as $\varepsilon_i = Y_i - \hat{Y_i}$

**Calculate the residuals and its summary statistics**

In [ ]:
# Calculate residuals
df['Residuals'] = df['SalePrice'] - df['SalePricePrediction']

In [ ]:
# Summary statistics

# Mean
mean = np.mean(df['Residuals'])
print(f"The estimated mean: {mean}")

# Median
median = np.median(df['Residuals'])
print(f"The estimated median: {median}")

# Standard deviation, ddof=1 ensures the maximum likelihood estimate
std = np.std(df['Residuals'], ddof = 1)
print(f"The estimated standard deviation is: {std}")

# Skewness
skewness = scipy.stats.skew(df['Residuals'])
print(f"The estimated skewness is: {skewness}")

# Kurtosis
kurtosis = scipy.stats.kurtosis(df['Residuals'])
print(f"The estimated kurtosis is: {kurtosis}")


**Investigate relationship between $Y_i$ and $\hat{Y_i}$ through a scatterplot**

In [ ]:
# Scatter plot
plt.scatter(df['SalePricePrediction'], df['SalePrice'])
plt.xlabel('Sale Price Prediction')
plt.ylabel('Sale Price')

plt.show()

**Investigate normality of residuals through histogram**

In [ ]:
# Histogram
(xmin, xmax) = (-100000,100000)
plt.hist(df['Residuals'], edgecolor='black', bins=30, range=(xmin, xmax), density=True)
plt.xlabel('Residuals')
plt.ylabel('Density')

# Fit a normal distribution
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, mean, std)

plt.plot(x, p, 'k', linewidth=2)
plt.show()

As we have already seen earlier, the kurtosis is too high. What happens if we apply a transformation: scaling the residuals by the prediction?

In [ ]:
# Calculate scaled residuals
df['ScaledResiduals'] = (df['SalePrice'] - df['SalePricePrediction']) / df['SalePricePrediction']
plt.xlabel('Scaled Residuals')
plt.ylabel('Density')


# Fit a normal distribution
(xmin, xmax) = (-1,1)
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, np.mean(df['ScaledResiduals']), np.std(df['ScaledResiduals']))

# Histogram
plt.hist(df['ScaledResiduals'], edgecolor='black', bins=30, range=(xmin, xmax), density=True)

plt.plot(x, p, 'k', linewidth=2)
plt.show()

These scaled residuals already look more like a normal distribution.

# Model selection, prediction intervals

**Estimate parameters of the following model using maximum likelihood estimation**

Note that by rewriting the normality assumption on the residuals, one can obtain model 1. <br>
Model 1: <br>
$Y_i \sim N\left(\hat{Y_i}, \sigma^2\right)$


In [ ]:
def neg_loglike_model1(theta):
    mu = df['SalePricePrediction']
    sd = theta[0]
    return -1*stats.norm(mu, sd).logpdf(df['SalePrice']).sum()


theta_start = np.array([1])

res = scipy.optimize.minimize(neg_loglike_model1, theta_start , method = 'Nelder-Mead',
           options={'disp': True})


sigma_mle1 = float(res['x'])
loglik1 = - res.fun
aic1 = 2*len(theta_start) - 2* loglik1

print(f"The numerical MLE for sigma: {sigma_mle1}")
print(f"The loglikelihood: {loglik1}")
print(f"The AIC: {aic1}")

As we have seen in the slides, given an estimated distribution function $\hat{F}$, we can use the inverse cumulative distribution function (quantile) $\hat{F}^{-1}$ to calculate the upper and lower prediction limits:<br>

$UPL = \hat{F}^{-1}(1-\alpha/2)$<br>
$LPL = \hat{F}^{-1}(\alpha/2)$<br>

Calculate the 95% prediction interval (i.e. $\alpha=0.05$) for $Y_0$ given $\hat{Y}_0$ based on the quantiles of model 1. Use the numerically estimated parameter.

In [ ]:
UPL1 = stats.norm.ppf(0.975, loc=df_0['SalePricePrediction'], scale=sigma_mle1)
LPL1 = stats.norm.ppf(0.025, loc=df_0['SalePricePrediction'], scale=sigma_mle1)

print(f"The predicted sale price is: {df_0['SalePricePrediction']}" )
print(f"With prediction interval:" )
print(LPL1, UPL1)


In this case we can verify that the real sale price $(Y_0 = €215.000)$ is within the prediction interval.

**Estimate parameters of the following models using maximum likelihood estimation**<br>
Other choices are possible to model the variance, for instance a variance which scales in the predicted sale price. <br>
Model 2: <br>
$Y_i \sim N\left(\hat{Y_i}, \left(\hat{Y_i}\sigma\right)^2\right)$

Model 3: <br>
$Y_i \sim N\left(\hat{Y_i}, \tau^2 + \left(\hat{Y_i}\sigma\right)^2\right)$


In [ ]:
def neg_loglike_model2(theta):
    mu = df['SalePricePrediction']
    sd = theta[0]*(df['SalePricePrediction'])
    return -1*stats.norm(mu, sd).logpdf(df['SalePrice']).sum()

theta_start = np.array([1])
res = scipy.optimize.minimize(neg_loglike_model2, theta_start, method = 'Nelder-Mead',
           options={'disp': True})
sigma_mle2 = float(res['x'])
loglik2 = - res.fun
aic2 = 2*len(theta_start) - 2* loglik2

print(f"The theoretical MLE for sigma: {np.std(df['SalePrice']/df['SalePricePrediction'])}")
print(f"The numerical MLE for sigma: {sigma_mle2}")
print(f"The loglikelihood: {loglik2}")
print(f"The AIC: {aic2}")

In [ ]:
def neg_loglike_model3(theta):
    mu = df['SalePricePrediction']
    sd = np.sqrt(theta[0]**2 + (theta[1]*df['SalePricePrediction'])**2)
    return -1*stats.norm(mu, sd).logpdf(df['SalePrice']).sum()

theta_start = np.array([1,1])
res = scipy.optimize.minimize(neg_loglike_model3, theta_start, method = 'Nelder-Mead',
           options={'disp': True})

tau_mle3 = float(res['x'][0])
sigma_mle3 = float(res['x'][1])

loglik3 = - res.fun
aic3 = 2*len(theta_start) - 2* loglik3

print(f"The numerical MLE for tau: {tau_mle3}")
print(f"The numerical MLE for sigma: {sigma_mle3}")
print(f"The loglikelihood: {loglik3}")
print(f"The AIC: {aic3}")

In [ ]:
y_0_hat = df_0['SalePricePrediction']
y_0 =  df_0['SalePrice']

# Prediction interval for model 2
UPL2 = stats.norm.ppf(0.975, loc=y_0_hat, scale=sigma_mle2*y_0_hat)
LPL2 = stats.norm.ppf(0.025, loc=y_0_hat, scale=sigma_mle2*y_0_hat)

# Prediction interval for model 3
UPL3 = stats.norm.ppf(0.975, loc=y_0_hat, scale=np.sqrt(tau_mle3**2 + (sigma_mle3*y_0_hat)**2))
LPL3 = stats.norm.ppf(0.025, loc=y_0_hat, scale=np.sqrt(tau_mle3**2 + (sigma_mle3*y_0_hat)**2))

print(f"The predicted sale price is: {y_0_hat}" )
print(f"Prediction interval of model 2:" )
print(np.round(LPL2), np.round(UPL2))

print(f"Prediction interval of model 3:" )
print(np.round(LPL3), np.round(UPL3))


We can now calculate prediction intervals given any predicted sale price according to the three methods:

In [ ]:
xx = np.arange(1,round(np.max(df['SalePricePrediction']))+1e5)

# Prediction intervals for model 1
upper_1 = stats.norm.ppf(0.975, loc=xx, scale=sigma_mle1)
lower_1 = stats.norm.ppf(0.025, loc=xx, scale=sigma_mle1)

# Prediction intervals for model 2
upper_2 = stats.norm.ppf(0.975, loc=xx, scale=sigma_mle2*xx)
lower_2 = stats.norm.ppf(0.025, loc=xx, scale=sigma_mle2*xx)

# Prediction intervals for model 3
upper_3 = stats.norm.ppf(0.975, loc=xx, scale=np.sqrt(tau_mle3**2 + (sigma_mle3*xx)**2))
lower_3 = stats.norm.ppf(0.025, loc=xx, scale=np.sqrt(tau_mle3**2 + (sigma_mle3*xx)**2))


fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(df['SalePricePrediction'], df['SalePrice'], "o", label="Data",alpha=0.7)
# Model 1
ax.plot(xx, upper_1, "r--", label="Model 1", linewidth= 2)
ax.plot(xx, lower_1, "r--",  linewidth= 2)


# Model 2
ax.plot(xx, upper_2, "k--", label="Model 2", linewidth= 2, alpha=.8)
ax.plot(xx, lower_2, "k--",  linewidth= 2, alpha=.8)

# Model 3
ax.plot(xx, upper_3, "b--", label="Model 3", linewidth= 2)
ax.plot(xx, lower_3, "b--",  linewidth= 2)

ax.set_xlim(1,round(np.max(df['SalePricePrediction']))+1e5)
ax.set_xlabel('Sale Price Prediction')
ax.set_ylabel('Sale Price')
ax.legend(loc="best")
fig.show()

But which model is the most adequate?

**Determine best model by comparing the AIC and applying the Likelihood Ratio Test.**

In [ ]:
print(f"The AIC of model 1: {aic1}")
print(f"The AIC of model 2: {aic2}")
print(f"The AIC of model 3: {aic3}")

print(f"The log-likelihood of model 1: {loglik1}")
print(f"The log-likelihood of model 2: {loglik2}")
print(f"The log-likelihood of model 3: {loglik3}")

Model 3 has the lowest AIC, implying that it is most adequate. <br>
Note that model 2 has a higher likelihood than model 1. Since their number of parameters is the same, model 2 outperforms model 1. Let us now compare model 2 and model 3 using the Likelihood Ratio Test.

In [ ]:
# Likelihood ratio test
L = -2*(loglik2 - loglik3)

# P value
p_val = 1 - stats.chi2.cdf(L, 1)

print(f"The likelihood ratio test statistic equals: {L}")
print(f"The corresponding p-value equals: {p_val}")

We conclude that model 3 is the most adequate model.

# Confidence intervals do not equal prediction intervals

Given our models, estimating a confidence interval for the mean of $Y_i$ is not interesting because it is assumed to equal exactly $\hat{Y}_i$. <br>

Therefore, in this simulation assume that data is normally distributed with mean 2 and standard deviation 1. Notice that the confidence interval of the mean, which is assumed to contain the real mean 95% of the time, is much smaller than the prediction interval, which is assumed to contain 95% of observtions.

In [ ]:
# Generate normally distributed data
np.random.seed(1)
true_mean = 2
data = np.random.normal(loc=true_mean, scale=1, size=100)

# Calculate the sample mean and standard deviation
sample_mean = np.mean(data)
sample_std = np.std(data, ddof=1)

# Set the confidence level
confidence_level = 0.95

# Calculate the confidence interval
z_critical = stats.norm.ppf(1 - (1 - confidence_level) / 2)
confidence_interval = z_critical * sample_std / np.sqrt(len(data))

# Calculate the prediction interval
pred_lower = stats.norm.ppf(1 - (1 - confidence_level) / 2, loc=sample_mean, scale=sample_std)
pred_upper = stats.norm.ppf((1 - confidence_level) / 2, loc=sample_mean, scale=sample_std)

# Plotting the histogram
fig, ax = plt.subplots(figsize=(8, 6))
ax.hist(data, bins=30, density=True, edgecolor='black', alpha=0.7, label='Data')

# Plotting the sample mean
ax.axvline(x=sample_mean, color='red', linestyle='--', label='Sample Mean')

# Plotting the confidence interval
ax.axvline(x=sample_mean - confidence_interval, color='green', linestyle='--', label='Confidence Interval')
ax.axvline(x=sample_mean + confidence_interval, color='green', linestyle='--')

# Plotting the prediction interval
ax.axvline(x=pred_lower, color='orange', linestyle='--', label='Prediction Interval')
ax.axvline(x=pred_upper, color='orange', linestyle='--')

ax.set_xlabel('Value')
ax.set_ylabel('Frequency')
ax.legend(loc='best')
plt.show()


****